# Face Recognition

Ahmed Amer, Nicholas Lee, Christian Valdez

we are using the ATT face database in the data folder

Three methods are implemented and compared in the following order:
- LBP
- Neural Networks
- SVM

## Importing Data

In [9]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

# Load the AT&T Faces dataset
data_folder = "./data/"
num_subjects = 40
num_images_per_subject = 10
image_size = (92, 112)

# Load the images and labels into arrays
images = []
labels = []
for subject_id in range(1, num_subjects + 1):
    for image_id in range(1, num_images_per_subject + 1):
        # Load the image and convert it to grayscale
        image_path = os.path.join(data_folder, "s" + str(subject_id), str(image_id) + ".pgm")
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, image_size)

        # Add the image and label to the arrays
        images.append(image)
        labels.append(subject_id)

# Convert the arrays to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert X_train to 32-bit floating-point numbers and normalize the pixel values
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

# Train an SVM classifier
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.train(X_train.reshape(-1, image_size[0]*image_size[1]), cv2.ml.ROW_SAMPLE, y_train)

# Make predictions on the test set
_, y_pred = svm.predict(X_test.reshape(-1, image_size[0]*image_size[1]))

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 100.00%
